In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [ ]:
import art
import asyncio
from dotenv import load_dotenv
import json
import openai
import random
import re
from typing import TypedDict
from openpipe.client import OpenPipe
import hashlib
import time

load_dotenv()

op_client = OpenPipe()
print("OpenPipe client initialized")


class TemporalCluePuzzle(TypedDict):
    num_clues: int
    prompt: str
    solution: dict[str, str]

def generate_puzzle_key(puzzle: TemporalCluePuzzle) -> str:
    # hash the puzzle prompt, truncate hex to 10 characters
    return str(puzzle["num_clues"]) + "-" + hashlib.sha256(puzzle["prompt"].encode()).hexdigest()[:10]


puzzles: list[TemporalCluePuzzle] = json.load(open("./data/temporal-clue/puzzles.json"))
val_puzzles = puzzles[:64]
test_puzzles = puzzles[64:128]
train_puzzles = puzzles[128:]
random.seed(42)
random.shuffle(train_puzzles)


api = art.LocalAPI(wandb_project="agent-reinforcement-training")
model = await api.get_or_create_model(
    name="temporal-clue-001", base_model="NousResearch/Hermes-2-Theta-Llama-3-8B"
)


async def rollout(
    client: openai.AsyncOpenAI, puzzle: TemporalCluePuzzle, iteration: int, is_validation: bool
) -> art.Trajectory:
    messages: art.Messages = [{"role": "user", "content": puzzle["prompt"]}]

    requested_at = int(time.time() * 1000)
    chat_completion = await client.chat.completions.create(
        messages=messages, model=model.name
    )

    puzzle_key = generate_puzzle_key(puzzle)
    print(f"puzzle_key: {puzzle_key}")
    op_client.report(
        requested_at=requested_at,
        received_at=int(time.time() * 1000),
        req_payload={
            "model": model.name,
            "messages": messages,
            "metadata": {
                "notebook-id": "temporal-clue",
                "iteration": str(iteration),
                "validation": str(is_validation),
                "puzzle_key": puzzle_key,
            },
        },
        resp_payload=chat_completion,
        status_code=200,
    )

    choice = chat_completion.choices[0]
    content = choice.message.content
    assert isinstance(content, str)
    num_correct = 0
    for key, value in puzzle["solution"].items():
        if matches := re.findall(rf"{key}\. ([A-Za-z \.:-]+)", content):
            match = matches[-1]
            if match.strip().lower() == value.lower():
                num_correct += 1
    reward = acc = num_correct / len(puzzle["solution"])
    return art.Trajectory(
        messages_and_choices=[*messages, choice], reward=reward, metrics={"acc": acc}
    )


stride = 32
for i in range(await model.get_iteration(), 10):
    async with model.openai_client(
        estimated_completion_tokens=350, verbosity=2
    ) as openai_client:
        val_groups, train_groups = await asyncio.gather(
            art.gather_trajectories(
                (
                    (rollout(openai_client, puzzle, i, is_validation=True) for _ in range(2))
                    for puzzle in val_puzzles
                ),
                pbar_desc="val",
                stream_chat_completions=8,
            ),
            art.gather_trajectories(
                (
                    (rollout(openai_client, puzzle, i, is_validation=False) for _ in range(50))
                    for puzzle in train_puzzles[i * stride : (i + 1) * stride]
                ),
                pbar_desc="train",
            ),
        )
    await model.log(val_groups)
    await model.clear_iterations()
    await model.tune(
        train_groups, config=art.TuneConfig(plot_tensors=True, verbosity=2)
    )